In [7]:
from main import main

import random
from copy import deepcopy

from shoe import Shoe
from game_config import Config
from hand import Hand, Dealer
from player import Player

In [ ]:
class Bot():
    def __init__(self, bankroll):
        self.bankroll = bankroll

In [8]:
bot = Bot(bankroll = 1000000)
shoe = Shoe(deck_quant = 8)
config = Config() # Define your configuration game here. Check game_config.py for reference

player = Player(bot.bankroll)

In [3]:
shoe = Shoe(deck_quant = 8)
config = Config() # Define your configuration game here. Check game_config.py for reference

card_quant = len(shoe.shoe) # Total cards in the shoe

bankroll = 1000000

player = Player(bankroll) # sign up Player
while player.balance > 0:
    bet = float(input('----------------- Betting Time ----------------- '))
    if bet <= player.balance:
        if config.dealt_first == 'p':
            player_hand, dealer_hand = shoe.deal_card(dealt_first = config.dealt_first)

            dealer = Dealer(dealer_hand)
            faceup_card = dealer.get_faceup_card()
            facedown_card = dealer.get_facedown_card()


            dealer_print = 'Dealer: [{} ; ? ] ({})'.format(
                faceup_card.__str__(), '/'.join([str(v) for v in faceup_card.get_score()])
                )


            player_hand = Hand(player_hand)

            print(dealer_print)
            print(hand_print(player_hand))

            while True: # Round tracking
                # Insurrance processing
                if faceup_card.card_face == 'Ac':
                    insurrance = input('Insurrance? (Y/N) ')

                    if insurrance.lower() == 'y': # Insurrance block
                        if dealer.blackjack_check():
                            player.win(bet, printout=False)
                        else:
                            player.lose(bet/2, printout=False)

                    if dealer.blackjack_check():
                        print('----------------------------------------------------')
                        if player_hand.blackjack_check(): # push on the main game
                            print(hand_print(dealer, player==False))
                            print(hand_print(player_hand))
                            player.tie(bet)
                        else:
                            print(hand_print(dealer, player==False))
                            print(hand_print(player_hand))
                            player.lose(bet)

                        break # End round

                # Blackjack processing
                if player_hand.blackjack_check():
                    print('----------------------------------------------------')
                    if dealer.blackjack_check():
                        print(hand_print(dealer, player==False))
                        print(hand_print(player_hand))
                        player.tie(bet)
                    else:
                        print(hand_print(dealer, player==False))
                        print(hand_print(player_hand))
                        player.win(bet*1.5)

                    break # End round

                else: # Player starts making decisions

                    hands = [player_hand]
                    choice_question = 'What is your choice? (X2, Hit, Stand or Split) '
                    hands_complete = []

                    hand_count = 0
                    while len(hands) != 0: # Keep playing for all hands you have
                        if hand_count != 0:
                            choice_question = 'Hand {} --- What is your choice? (X2, Hit, Stand or Split) '.format(hand_count)
                        hand_in_play = deepcopy(hands[0])
                        hand_delete = False
                        # Player making decisions
                        while hand_in_play.get_printable_score() not in ['21','']: # Player stops making decisions if the hand is either Bj, 21 or Busted

                            choice = input(choice_question)

                            if choice.lower() == 'x2' and hand_in_play.action == 0 and player_hand.action == 0: # Switch to play_in_hand if you allow x2 after splitting
                                if bet*2 <= player.balance:
                                    bet = bet*2
                                    hand_in_play.add_card(shoe.hit())
                                    break # End decision block
                                else:
                                    print('Your balance is too low')	
                            elif choice.lower() == 'hit':
                                hand_in_play.add_card(shoe.hit())
                                print(dealer_print)
                                print(hand_print(hand_in_play))
                            elif choice.lower() == 'stand':
                                break # End decision block
                            elif choice.lower() == 'split' and player_hand.split_check() == True: # Switch to play_in_hand if you allow to split multiple times
                                if player.balance >= bet*2:
                                    hand_1, hand_2 = player_hand.split(shoe)
                                    print(hand_print(hand_1, hand_num = 1, bj_check = False))
                                    print(hand_print(hand_2, hand_num = 2, bj_check = False))
                                    hands += [hand_1, hand_2]
                                    hand_delete = True
                                    break # End decision block
                                else:
                                    print('Your balance is too low')
                            else:	
                                print('You cannot {}'.format(choice.title()))

                        hands.pop(0) # delete the current hand

                        if hand_delete == False:
                            hands_complete.append(hand_in_play)

                        hand_count += 1

                        print('----------------------------------------------------')

                    # # Handling results after decisions making
                    # for hand_played in hands_complete:
                    # 	if hand_played.busted_check(): # If you busted, you lose right away
                    # 		# print(hand_print(dealer, player==False))
                    # 		print(hand_print(hand_played))


                    if all([v.busted_check() for v in hands_complete]) or dealer.blackjack_check():
                        print(hand_print(dealer, player=False))
                        player.lose(bet*len(hands_complete))

                        break # End round

                    else:
                        while dealer.get_max_score() < 17: # Dealer takes actions
                            dealer.add_card(shoe.hit())

                        dealer_score = dealer.get_max_score() # Dealer's score
                        for hand_played in hands_complete:

                            player_score = hand_played.get_max_score()

                            print(hand_print(dealer, player==False))
                            if player_score > 21:
                                print(hand_print(hand_played))
                                player.lose(bet)

                            elif (dealer_score > 21) or (dealer_score < player_score): 

                                # print(hand_print(dealer, player==False))
                                print(hand_print(hand_played, bj_check = False))
                                player.win(bet)

                            elif dealer_score > player_score:

                                # print(hand_print(dealer, player==False))
                                print(hand_print(hand_played, bj_check = False))
                                player.lose(bet)

                            elif dealer_score == player_score:

                                # print(hand_print(dealer, player==False))
                                print(hand_print(hand_played, bj_check = False))
                                player.tie(bet)

                        break # End round

    else:
        print('Betting money exceeds your bankroll')

    if len(shoe.shoe) < int(card_quant*config.reset_shoe):
        shoe.shuffle(reset=True)
        print('----------------- Shuffle Time -----------------')


In [6]:
shoe.get_shoe()

['Ja:D',
 'Ja:D',
 'Ja:H',
 '09:D',
 '10:S',
 '05:C',
 '05:D',
 '07:H',
 'Ja:H',
 '05:D',
 '10:H',
 'Ac:S',
 '06:D',
 'Ki:S',
 '04:D',
 '07:S',
 '03:C',
 '10:C',
 'Ja:S',
 'Ac:C',
 '09:C',
 '07:H',
 'Ki:D',
 '07:S',
 '05:H',
 '03:H',
 '03:S',
 '03:C',
 'Ki:D',
 '07:H',
 '08:S',
 '10:D',
 'Ja:C',
 'Ja:C',
 'Ki:D',
 '05:D',
 '09:H',
 'Ki:D',
 '02:C',
 '07:C',
 'Ac:C',
 '09:D',
 '02:H',
 'Qe:H',
 'Ki:H',
 'Ja:D',
 '09:C',
 '03:H',
 'Ki:S',
 '02:C',
 'Ac:D',
 '05:S',
 '05:H',
 '05:C',
 '02:D',
 '04:C',
 'Ac:D',
 '08:S',
 'Qe:D',
 'Ac:H',
 '08:S',
 '03:H',
 '02:H',
 '03:C',
 '05:S',
 '08:D',
 'Ja:C',
 'Ac:D',
 '04:C',
 'Ja:S',
 '05:H',
 '08:C',
 'Qe:C',
 '03:S',
 'Ja:H',
 '04:H',
 '03:S',
 '08:C',
 '10:C',
 '04:S',
 'Qe:C',
 '06:C',
 'Ja:C',
 '08:D',
 'Ja:C',
 '03:D',
 '08:H',
 'Qe:S',
 '02:D',
 '07:C',
 'Ja:S',
 'Ki:C',
 '10:H',
 'Ac:C',
 'Qe:C',
 '06:H',
 '10:H',
 '09:C',
 '03:H',
 '06:S',
 '05:C',
 'Ac:S',
 '07:C',
 'Ja:S',
 'Ki:H',
 'Ki:C',
 '04:D',
 '06:C',
 '02:C',
 '02:S',
 'Ki:S',
 

In [3]:
bj.deck()

TypeError: 'int' object is not callable

In [6]:
deck = []
card_num = np.arange(1,14)
card_type = ['B','T','R','C']
for v in card_num:
    for _v in card_type:
        deck.append(str(v) + _v)

['1B',
 '1T',
 '1R',
 '1C',
 '2B',
 '2T',
 '2R',
 '2C',
 '3B',
 '3T',
 '3R',
 '3C',
 '4B',
 '4T',
 '4R',
 '4C',
 '5B',
 '5T',
 '5R',
 '5C',
 '6B',
 '6T',
 '6R',
 '6C',
 '7B',
 '7T',
 '7R',
 '7C',
 '8B',
 '8T',
 '8R',
 '8C',
 '9B',
 '9T',
 '9R',
 '9C',
 '10B',
 '10T',
 '10R',
 '10C',
 '11B',
 '11T',
 '11R',
 '11C',
 '12B',
 '12T',
 '12R',
 '12C',
 '13B',
 '13T',
 '13R',
 '13C']